In [1]:
import os
import glob
import pandas as pd 
import random
import Levenshtein
import numpy as np
from scipy.io import savemat

# กำหนด paths
path_benign = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Benign"
path_malware = "C:\\Users\\BMEI CMU\\Documents\\DikeASCII_DATASET\\Malware"

# ฟังก์ชันในการอ่านไฟล์ .txt และแทนที่ช่องว่างด้วย '0' และรวมข้อมูลเป็นแถวเดียว พร้อมเพิ่มคอลัมน์เลข
def read_and_replace_spaces_as_rows(folder_path, label):
    # ค้นหาไฟล์ .txt ทั้งหมดในโฟลเดอร์และเลือกเฉพาะไฟล์ที่ต้องการ
    file_paths = glob.glob(os.path.join(folder_path, "*.txt"))
    modified_data_rows = []
    
    for file_path in file_paths:
        # อ่านเนื้อหาจากไฟล์แต่ละบรรทัดและรวมบรรทัดทั้งหมดไว้ในแถวเดียว
        with open(file_path, 'r', encoding='utf-8') as file:
            data = file.read().replace('\n', '')  # รวมบรรทัดทั้งหมดเป็นแถวเดียว
        
        # แทนที่ช่องว่างด้วย '0' ในแถวเดียว
        modified_data = ''.join(['0' if char == ' ' else char for char in data])
        
        # เพิ่มข้อมูลเป็น dictionary พร้อม label เพื่อแสดงเป็นคอลัมน์
        modified_data_rows.append({"data": modified_data, "label": label})

    return modified_data_rows

# อ่านและแทนที่ช่องว่างในไฟล์จากโฟลเดอร์ benign และ malware พร้อม label
benign_data = read_and_replace_spaces_as_rows(path_benign, 0)
malware_data = read_and_replace_spaces_as_rows(path_malware, 1)

# แปลงข้อมูลเป็น DataFrame แยกกัน
benign_df = pd.DataFrame(benign_data)
malware_df = pd.DataFrame(malware_data)

In [ ]:
# กำหนดจำนวนแถวที่จะสุ่ม
num_samples_benign = 1000
num_samples_malware = 10000

# สุ่มเลือกแถวจาก benign_data พร้อมเก็บ index ของแถวที่เลือก
benign_sample_indices = random.sample(range(len(benign_df)), num_samples_benign)
benign_train_data = benign_df.iloc[benign_sample_indices].reset_index(drop=True)

# สุ่มเลือกแถวจาก malware_data พร้อมเก็บ index ของแถวที่เลือก
malware_sample_indices = random.sample(range(len(malware_df)), num_samples_malware)
malware_train_data = malware_df.iloc[malware_sample_indices].reset_index(drop=True)

# สร้าง DataFrame สำหรับ benign และ malware ที่เหลือเป็น TEST โดยใช้ดัชนีที่ไม่ได้ถูกสุ่มมา
benign_test_df = benign_df.drop(benign_sample_indices).reset_index(drop=True)
malware_test_df = malware_df.drop(malware_sample_indices).reset_index(drop=True)

# กำหนด path สำหรับบันทึก benign และ malware TRAIN และ TEST
benign_train_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\benign_train.json"
malware_train_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_train.json"
benign_test_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\benign_test.json"
malware_test_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_test.json"
benign_indices_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\benign_sample_indices.txt"
malware_indices_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_sample_indices.txt"

# บันทึก benign และ malware TRAIN เป็นไฟล์ JSON
benign_train_data.to_json(benign_train_path, orient="records", lines=True)
malware_train_data.to_json(malware_train_path, orient="records", lines=True)

# บันทึก benign และ malware TEST เป็นไฟล์ JSON
benign_test_df.to_json(benign_test_path, orient="records", lines=True)
malware_test_df.to_json(malware_test_path, orient="records", lines=True)

# บันทึก benign_sample_indices และ malware_sample_indices เป็นไฟล์ .txt
with open(benign_indices_path, 'w') as f:
    f.write("\n".join(map(str, benign_sample_indices)))

with open(malware_indices_path, 'w') as f:
    f.write("\n".join(map(str, malware_sample_indices)))


In [4]:
# กำหนดจำนวนแถวที่จะสุ่ม
num_samples_malware = 1000

# สุ่มเลือกแถวจาก malware_data พร้อมเก็บ index ของแถวที่เลือก
malware_sample_indices = random.sample(range(len(malware_df)), num_samples_malware)
malware_train_data = malware_df.iloc[malware_sample_indices].reset_index(drop=True)

# กำหนด path สำหรับบันทึก benign และ malware TRAIN และ TEST
malware_train_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_train_1000.json"
malware_indices_path = r"C:\Users\BMEI CMU\Documents\GitHub\WORK\Windows\CODE_BME\PROJECT_CYBER_SECURITY\RESULT\01.TRAIN_TEST_SET\malware_sample_indices_1000.txt"

# บันทึก benign และ malware TRAIN เป็นไฟล์ JSON
malware_train_data.to_json(malware_train_path, orient="records", lines=True)

# บันทึก benign_sample_indices และ malware_sample_indices เป็นไฟล์ .txt
with open(malware_indices_path, 'w') as f:
    f.write("\n".join(map(str, malware_sample_indices)))